In [57]:
import os
import nltk
from nltk import FreqDist
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [43]:
df = pd.read_json('data.json')

In [69]:
df['name']

0         t-mobile-atlanta-13
1          t-mobile-atlanta-3
2         t-mobile-atlanta-39
3            t-mobile-austell
4          t-mobile-lilburn-2
5         t-mobile-atlanta-30
6         t-mobile-atlanta-23
7         t-mobile-atlanta-46
8     t-mobile-lithia-springs
9          t-mobile-atlanta-7
10        t-mobile-east-point
11        t-mobile-atlanta-35
12        t-mobile-atlanta-28
13      t-mobile-alpharetta-7
14        t-mobile-atlanta-43
Name: name, dtype: object

In [83]:
df.loc[df['name'] == 't-mobile-atlanta-30']['reviews'].array

<PandasArray>
[[['I had a great experience switching from AT&T to T-Mobile and buying my 1st iPhone. Trey and Shamika were helpful, as well as fun. There was a glitch and we were unable to transfer contacts as well as photos to my new phone. Granted, my old phone was a Nokia Lumia & that was a challenge.', 4.0, '\n        2/7/2019\n    '], ['Wasted about an hour visiting this store that I won\'t get back. Staff seemed confused on how to address the problems I had, misleading me about certain things that I later found out on my own not to be the case after speaking with the call center. On top of that, the staff was rude as hell and it seemed like they just didn\'t want to be there. While at the store and speaking with a representative over the phone trying to assist the situation, I tried putting him in contact with the gentleman at the store assisting me, to which the gentleman immediately got defensive and said "I don\'t need to talk to them. I can call them on my own if I wanted to 

In [68]:
subscription_key = 'c1005ffa18cc42ddbf916d2483bb2dd2'
assert subscription_key
text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"

environ({'TERM_PROGRAM_VERSION': '421', 'LOGNAME': 'jeffreyjacob', 'USER': 'jeffreyjacob', 'PATH': '/Library/Frameworks/Python.framework/Versions/3.7/bin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/share/dotnet:/opt/X11/bin:/Library/Frameworks/Mono.framework/Versions/Current/Commands:/Applications/Wireshark.app/Contents/MacOS:/usr/local/git/bin:/Applications/Xamarin Workbooks.app/Contents/SharedSupport/path-bin', 'HOME': '/Users/jeffreyjacob', 'DISPLAY': '/private/tmp/com.apple.launchd.YDKp4qa6eO/org.macosforge.xquartz:0', 'TERM_PROGRAM': 'Apple_Terminal', 'LANG': 'en_US.UTF-8', 'TERM': 'xterm-color', 'Apple_PubSub_Socket_Render': '/private/tmp/com.apple.launchd.HZ5YnvhRQd/Render', 'SHLVL': '1', 'XPC_FLAGS': '0x0', 'TMPDIR': '/var/folders/2d/chc6k12d29dbdkdy3lg60d180000gn/T/', 'TERM_SESSION_ID': '79F44D52-2469-4D36-870E-9534F87EC16C', 'XPC_SERVICE_NAME': '0', 'JPY_PARENT_PID': '16814', 'SSH_AUTH_SOCK'

KeyError: 'MS_KEY'

In [55]:
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()
  
  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
  
  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  plt.figure(figsize=(20,5))
  ax = sns.barplot(data=d, x= "word", y = "count")
  ax.set(ylabel = 'Count')
  plt.show()

In [72]:
for i in range(len(df)):
    
    idnum = 1
    docDic = []
    for rev in raw_revs:
        docDic.append({'id':idnum, 'language':'en', 'text': rev[0]})
        idnum += 1
        
    documents = {'documents' : docDic}
    key_phrase_api_url = text_analytics_base_url + "keyPhrases"
    headers   = {'Ocp-Apim-Subscription-Key': subscription_key}
    response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
    key_phrases = response.json()
    
    sentiment_api_url = text_analytics_base_url + "sentiment"
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    
    entity_linking_api_url = text_analytics_base_url + "entities"
    response  = requests.post(entity_linking_api_url, headers=headers, json=documents)
    entities = response.json()
    pprint(entities)
    
    keyphr = []
    senti = []
    try:
        for i in range(len(key_phrases['documents'])):
            keyphr += key_phrases['documents'][i]['keyPhrases']
            senti += [sentiments['documents'][i]['score']]
    except:
        pprint(key_phrases)
    
#     freq_words(keyphr)
    
    currStore = df.loc[i]['name']
    print("Store: " + currStore)
    print("Loc: " + str(i))
    raw_revs = df.loc[i]['reviews']
    scores = [rev[1] for rev in raw_revs]
    print ("Average Score: %f" % (sum(scores) / float(len(scores))))

#     if (sum(senti) / float(len(senti)) > 0.5):
#         print("HAPPY")
#     else:
#         print("BAD")
    

{'documents': [{'entities': [{'bingId': '3d449f2f-d7b9-2425-7362-f5e131098910',
                              'matches': [{'length': 8,
                                           'offset': 15,
                                           'text': 'T-Mobile'}],
                              'name': 'T-Mobile US',
                              'wikipediaId': 'T-Mobile US',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/T-Mobile_US'}],
                'id': '1'},
               {'entities': [{'bingId': '74604c1f-6f14-9867-25ce-ea69e540c8d8',
                              'matches': [{'length': 9,
                                           'offset': 34,
                                           'text': 'Employees'}],
                              'name': 'Employment',
                              'wikipediaId': 'Employment',
                              'wikipediaLanguage': 'en',
                        

{'documents': [{'entities': [{'bingId': 'aabdfc85-5c81-5ece-793f-f5d9358040ba',
                              'matches': [{'length': 8,
                                           'offset': 715,
                                           'text': 'SIM card'},
                                          {'length': 8,
                                           'offset': 468,
                                           'text': 'SIM card'},
                                          {'length': 8,
                                           'offset': 597,
                                           'text': 'SIM card'}],
                              'name': 'Subscriber identity module',
                              'wikipediaId': 'Subscriber identity module',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Subscriber_identity_module'},
                             {'bingId': '3d449f2f-d7b9-2425-7362-f5e131098910'

{'documents': [{'entities': [{'bingId': '3313fc64-e91d-4320-463a-014fa65344b8',
                              'matches': [{'length': 11,
                                           'offset': 252,
                                           'text': 'Nokia Lumia'}],
                              'name': 'Microsoft Lumia',
                              'wikipediaId': 'Microsoft Lumia',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Microsoft_Lumia'},
                             {'bingId': '3d449f2f-d7b9-2425-7362-f5e131098910',
                              'matches': [{'length': 8,
                                           'offset': 48,
                                           'text': 'T-Mobile'}],
                              'name': 'T-Mobile US',
                              'wikipediaId': 'T-Mobile US',
                              'wikipediaLanguage': 'en',
                              'wiki

{'documents': [{'entities': [{'bingId': '480b9330-1f17-8cbf-7469-bf9f560decf4',
                              'matches': [{'length': 20,
                                           'offset': 235,
                                           'text': 'I-85 bridge collapse'}],
                              'name': 'Interstate 85 bridge collapse',
                              'wikipediaId': 'Interstate 85 bridge collapse',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Interstate_85_bridge_collapse'},
                             {'bingId': '75d2893f-8181-2598-e5a7-73efc711e32a',
                              'matches': [{'length': 14,
                                           'offset': 608,
                                           'text': 'Chinese buffet'}],
                              'name': 'American Chinese cuisine',
                              'wikipediaId': 'American Chinese cuisine',
       

{'documents': [{'entities': [{'bingId': 'c9826276-77d7-c446-98c8-0d251f27cd70',
                              'matches': [{'length': 14,
                                           'offset': 98,
                                           'text': 'Perimeter Mall'}],
                              'name': 'Perimeter Mall',
                              'wikipediaId': 'Perimeter Mall',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Perimeter_Mall'},
                             {'bingId': 'db63aea8-3799-393d-678a-5f62c300d23d',
                              'matches': [{'length': 10,
                                           'offset': 343,
                                           'text': 'MyTouch 4G'}],
                              'name': 'T-Mobile myTouch 4G',
                              'wikipediaId': 'T-Mobile myTouch 4G',
                              'wikipediaLanguage': 'en',
                

{'documents': [{'entities': [{'bingId': '480b9330-1f17-8cbf-7469-bf9f560decf4',
                              'matches': [{'length': 20,
                                           'offset': 235,
                                           'text': 'I-85 bridge collapse'}],
                              'name': 'Interstate 85 bridge collapse',
                              'wikipediaId': 'Interstate 85 bridge collapse',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Interstate_85_bridge_collapse'},
                             {'bingId': '75d2893f-8181-2598-e5a7-73efc711e32a',
                              'matches': [{'length': 14,
                                           'offset': 608,
                                           'text': 'Chinese buffet'}],
                              'name': 'American Chinese cuisine',
                              'wikipediaId': 'American Chinese cuisine',
       

{'documents': [{'entities': [{'bingId': 'c9826276-77d7-c446-98c8-0d251f27cd70',
                              'matches': [{'length': 14,
                                           'offset': 98,
                                           'text': 'Perimeter Mall'}],
                              'name': 'Perimeter Mall',
                              'wikipediaId': 'Perimeter Mall',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Perimeter_Mall'},
                             {'bingId': 'db63aea8-3799-393d-678a-5f62c300d23d',
                              'matches': [{'length': 10,
                                           'offset': 343,
                                           'text': 'MyTouch 4G'}],
                              'name': 'T-Mobile myTouch 4G',
                              'wikipediaId': 'T-Mobile myTouch 4G',
                              'wikipediaLanguage': 'en',
                

{'documents': [{'entities': [{'bingId': '480b9330-1f17-8cbf-7469-bf9f560decf4',
                              'matches': [{'length': 20,
                                           'offset': 235,
                                           'text': 'I-85 bridge collapse'}],
                              'name': 'Interstate 85 bridge collapse',
                              'wikipediaId': 'Interstate 85 bridge collapse',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Interstate_85_bridge_collapse'},
                             {'bingId': '75d2893f-8181-2598-e5a7-73efc711e32a',
                              'matches': [{'length': 14,
                                           'offset': 608,
                                           'text': 'Chinese buffet'}],
                              'name': 'American Chinese cuisine',
                              'wikipediaId': 'American Chinese cuisine',
       

{'documents': [{'entities': [{'bingId': 'c9826276-77d7-c446-98c8-0d251f27cd70',
                              'matches': [{'length': 14,
                                           'offset': 98,
                                           'text': 'Perimeter Mall'}],
                              'name': 'Perimeter Mall',
                              'wikipediaId': 'Perimeter Mall',
                              'wikipediaLanguage': 'en',
                              'wikipediaUrl': 'https://en.wikipedia.org/wiki/Perimeter_Mall'},
                             {'bingId': 'db63aea8-3799-393d-678a-5f62c300d23d',
                              'matches': [{'length': 10,
                                           'offset': 343,
                                           'text': 'MyTouch 4G'}],
                              'name': 'T-Mobile myTouch 4G',
                              'wikipediaId': 'T-Mobile myTouch 4G',
                              'wikipediaLanguage': 'en',
                

{'message': 'Rate limit is exceeded. Try again in 46 seconds.',
 'statusCode': 429}
{'message': 'Rate limit is exceeded. Try again in 47 seconds.',
 'statusCode': 429}
Store: t-mobile-east-point
Loc: 10
Average Score: 1.833333
{'message': 'Rate limit is exceeded. Try again in 46 seconds.',
 'statusCode': 429}
{'message': 'Rate limit is exceeded. Try again in 46 seconds.',
 'statusCode': 429}
Store: t-mobile-atlanta-35
Loc: 11
Average Score: 4.000000
{'message': 'Rate limit is exceeded. Try again in 45 seconds.',
 'statusCode': 429}
{'message': 'Rate limit is exceeded. Try again in 45 seconds.',
 'statusCode': 429}
Store: t-mobile-atlanta-28
Loc: 12
Average Score: 2.600000
{'message': 'Rate limit is exceeded. Try again in 44 seconds.',
 'statusCode': 429}
{'message': 'Rate limit is exceeded. Try again in 45 seconds.',
 'statusCode': 429}
Store: t-mobile-alpharetta-7
Loc: 13
Average Score: 3.200000
{'message': 'Rate limit is exceeded. Try again in 43 seconds.',
 'statusCode': 429}
{'mess